In [64]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=67bf9cf8a2882501ac3a591b159be390c85b9d645921f475caeab6146242b1af
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [65]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta
import openpyxl
import xlrd
import os

import warnings
warnings.filterwarnings('ignore')

In [66]:
import setuptools

In [67]:
# pytorch library
import torch # the main pytorch library
import torch.nn.functional as f # the sub-library containing different functions for manipulating with tensors

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

In [68]:
# Code to read csv file into Colaboratory:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [69]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [70]:
path_in = '/drive/MyDrive/Doutorado_v2/'
df_file = 'E03_file_cluster_full_v01.xlsx'
file_in = os.path.join(path_in, df_file)
DF = pd.read_excel(file_in)
DF.shape

(15989, 19)

In [71]:
DF.dtypes

name_file                      object
data_noticia_corrig    datetime64[ns]
Ano                             int64
Mes                             int64
Contexto                       object
silhouette_score              float64
Cluster Label                   int64
PCA1                          float64
PCA2                          float64
Termo_01                       object
Termo_02                       object
Termo_03                       object
Termo_04                       object
Termo_05                       object
Termo_06                       object
Termo_07                       object
Termo_08                       object
Termo_09                       object
Termo_10                       object
dtype: object

In [72]:
def binary_map(feature):
    return feature.map({0:1, 1:2, 2:3, 3:4, 4:5})

## Encoding target feature
DF['Cluster Label'] = DF[['Cluster Label']].apply(binary_map)

In [73]:
DF.head()

,name_file,data_noticia_corrig,Ano,Mes,Contexto,silhouette_score,Cluster Label,PCA1,PCA2,Termo_01,Termo_02,Termo_03,Termo_04,Termo_05,Termo_06,Termo_07,Termo_08,Termo_09,Termo_10
0,3809.pdf,2020-03-10,2020,0,Geral,0.055167,1,-0.074987,-0.006647,mineração,empresa,brasil,vale,minas,projeto,mineradora,setor,mining,mina
1,2749.pdf,2020-06-16,2020,0,Geral,0.055167,1,-0.071070,-0.006029,mineração,empresa,brasil,vale,minas,projeto,mineradora,setor,mining,mina
2,3635.pdf,2020-01-30,2020,0,Geral,0.055167,1,-0.073000,-0.006273,mineração,empresa,brasil,vale,minas,projeto,mineradora,setor,mining,mina
3,2849.pdf,2020-06-30,2020,0,Geral,0.055167,1,-0.082184,-0.008022,mineração,empresa,brasil,vale,minas,projeto,mineradora,setor,mining,mina
4,3594.pdf,2017-07-25,2017,0,Geral,0.055167,1,-0.078308,-0.007974,mineração,empresa,brasil,vale,minas,projeto,mineradora,setor,mining,mina


In [74]:
DF['Cluster Label'].unique()

array([1, 2, 4, 5, 3])

In [75]:
DF['Contexto'].unique()

array(['Geral', 'Sem similaridade', 'Geral por ano',
       'Sem Similaridade por ano', 'Geral por mês',
       'Sem Similaridade por mês'], dtype=object)

In [76]:
DF = DF[DF['Contexto'] == 'Sem Similaridade por ano']
DF.shape

(2315, 19)

In [77]:
DF.columns

Index(['name_file', 'data_noticia_corrig', 'Ano', 'Mes', 'Contexto',
       'silhouette_score', 'Cluster Label', 'PCA1', 'PCA2', 'Termo_01',
       'Termo_02', 'Termo_03', 'Termo_04', 'Termo_05', 'Termo_06', 'Termo_07',
       'Termo_08', 'Termo_09', 'Termo_10'],
      dtype='object')

In [78]:
DF = DF.groupby(['Contexto', 'Ano', 'Cluster Label', 'Termo_01', 'Termo_02', 'Termo_03', 'Termo_04', 'Termo_05', 'Termo_06', 'Termo_07', 'Termo_08', 'Termo_09', 'Termo_10']).agg(N = ('name_file', 'count')).reset_index()
DF.shape

(45, 14)

In [79]:
DF['Termos'] = DF[['Termo_01', 'Termo_02', 'Termo_03', 'Termo_04', 'Termo_05', 'Termo_06', 'Termo_07', 'Termo_08', 'Termo_09', 'Termo_10']].agg(', '.join, axis=1)

In [80]:
DF.head()

,Contexto,Ano,Cluster Label,Termo_01,Termo_02,Termo_03,Termo_04,Termo_05,Termo_06,Termo_07,Termo_08,Termo_09,Termo_10,N,Termos
0,Sem Similaridade por ano,2013,1,empresa,mineração,ouro,projeto,marco,ano,anos,belo,mining,país,33,"empresa, mineração, ouro, projeto, marco, ano,..."
1,Sem Similaridade por ano,2013,2,ogx,queda,mercado,ações,ibovespa,presse,baixa,the,índice,recuo,37,"ogx, queda, mercado, ações, ibovespa, presse, ..."
2,Sem Similaridade por ano,2013,3,libertação,eln,tibete,exército,bogotá,solar,wober,dias,serviços,nacional,5,"libertação, eln, tibete, exército, bogotá, sol..."
3,Sem Similaridade por ano,2013,4,mmx,minério,toneladas,ferro,recursos,azul,serra,bilhões,milhões,sudeste,15,"mmx, minério, toneladas, ferro, recursos, azul..."
4,Sem Similaridade por ano,2013,5,ibovespa,alta,valorização,ganhos,brasília,horário,pontos,ações,shares,forte,26,"ibovespa, alta, valorização, ganhos, brasília,..."


In [81]:
DF.tail()

,Contexto,Ano,Cluster Label,Termo_01,Termo_02,Termo_03,Termo_04,Termo_05,Termo_06,Termo_07,Termo_08,Termo_09,Termo_10,N,Termos
40,Sem Similaridade por ano,2021,1,mineração,mmx,administração,conselho,operações,publicado,companhia,estado,vale,ouro,75,"mineração, mmx, administração, conselho, opera..."
41,Sem Similaridade por ano,2021,2,trimestre,minério,csn,milhões,ferro,ano,preços,mineração,toneladas,tonelada,96,"trimestre, minério, csn, milhões, ferro, ano, ..."
42,Sem Similaridade por ano,2021,3,vale,bilhões,mineradora,empresa,ações,acordo,oferta,ano,mineração,companhia,110,"vale, bilhões, mineradora, empresa, ações, aco..."
43,Sem Similaridade por ano,2021,4,mineração,energia,empresa,setor,brasil,áreas,projeto,meio,anos,minas,194,"mineração, energia, empresa, setor, brasil, ár..."
44,Sem Similaridade por ano,2021,5,índice,pontos,ações,bolsa,alta,tóquio,fechou,queda,europeias,valores,31,"índice, pontos, ações, bolsa, alta, tóquio, fe..."


In [82]:
teste = DF.groupby(['Termos']).agg(N = ('Contexto', 'count')).reset_index()
teste = teste[teste['N'] > 1]
teste

,Termos,N


In [83]:
DF2 = DF[['Termos']]
DF2['N'] = 1
DF2 = pd.merge(DF2, DF2, on='N', suffixes=['','2']).query("Termos != Termos2")
DF2 = DF2[['Termos', 'Termos2', 'N']]
DF2 = DF2.rename(columns={'Termos': 'Col_X', 'Termos2': 'Col_Y'}).reset_index()

In [84]:
DF2.shape

(1980, 4)

In [85]:
DF2.head(10)

,index,Col_X,Col_Y,N
0,1,"empresa, mineração, ouro, projeto, marco, ano,...","ogx, queda, mercado, ações, ibovespa, presse, ...",1
1,2,"empresa, mineração, ouro, projeto, marco, ano,...","libertação, eln, tibete, exército, bogotá, sol...",1
2,3,"empresa, mineração, ouro, projeto, marco, ano,...","mmx, minério, toneladas, ferro, recursos, azul...",1
3,4,"empresa, mineração, ouro, projeto, marco, ano,...","ibovespa, alta, valorização, ganhos, brasília,...",1
4,5,"empresa, mineração, ouro, projeto, marco, ano,...","ibovespa, alta, desvalorização, mmx, valorizaç...",1
5,6,"empresa, mineração, ouro, projeto, marco, ano,...","ano, empresa, mineração, ouro, milhões, projet...",1
6,7,"empresa, mineração, ouro, projeto, marco, ano,...","ibovespa, fell, list, ibovespas, shares, losse...",1
7,8,"empresa, mineração, ouro, projeto, marco, ano,...","minério, ferro, preço, disse, produção, compan...",1
8,9,"empresa, mineração, ouro, projeto, marco, ano,...","projetos, empresa, mil, brasil, mineração, our...",1
9,10,"empresa, mineração, ouro, projeto, marco, ano,...","jaguar, trimestre, ouro, ano, onças, sonhos, c...",1


In [86]:
DF2.tail()

,index,Col_X,Col_Y,N
1975,2019,"índice, pontos, ações, bolsa, alta, tóquio, fe...","tóquio, fechou, registradas, valores, bolsa, m...",1
1976,2020,"índice, pontos, ações, bolsa, alta, tóquio, fe...","mineração, mmx, administração, conselho, opera...",1
1977,2021,"índice, pontos, ações, bolsa, alta, tóquio, fe...","trimestre, minério, csn, milhões, ferro, ano, ...",1
1978,2022,"índice, pontos, ações, bolsa, alta, tóquio, fe...","vale, bilhões, mineradora, empresa, ações, aco...",1
1979,2023,"índice, pontos, ações, bolsa, alta, tóquio, fe...","mineração, energia, empresa, setor, brasil, ár...",1


In [87]:
i = 1
DF2.iloc[i]['Col_X']

'empresa, mineração, ouro, projeto, marco, ano, anos, belo, mining, país'

In [88]:
for i in range(0, len(DF2)):

   text11 = DF2.iloc[i]['Col_X']
   text22 = DF2.iloc[i]['Col_Y']

   texts = [text11, text22]

   texts_embeddings = model.encode(texts)

   a = cosine_similarity([texts_embeddings[0]], texts_embeddings[1:])

   DF2.loc[i, 'Similarity'] = a[0,0]

In [91]:
DF2['Fx_Similarity'] = np.where(DF2['Similarity'] >= 0.95, 'Perfeita',
                       np.where(DF2['Similarity'] >= 0.75, 'Forte',
                       np.where(DF2['Similarity'] >= 0.5, 'Moderada',
                       np.where(DF2['Similarity'] >= 0.25, 'Fraca',
                       np.where(DF2['Similarity'] >= 0.1, 'Ínfima', 'Nula')))))

In [92]:
DF2.head()

,index,Col_X,Col_Y,N,Similarity,Fx_Similarity
0,1,"empresa, mineração, ouro, projeto, marco, ano,...","ogx, queda, mercado, ações, ibovespa, presse, ...",1,0.718788,Moderada
1,2,"empresa, mineração, ouro, projeto, marco, ano,...","libertação, eln, tibete, exército, bogotá, sol...",1,0.735246,Moderada
2,3,"empresa, mineração, ouro, projeto, marco, ano,...","mmx, minério, toneladas, ferro, recursos, azul...",1,0.871109,Forte
3,4,"empresa, mineração, ouro, projeto, marco, ano,...","ibovespa, alta, valorização, ganhos, brasília,...",1,0.720575,Moderada
4,5,"empresa, mineração, ouro, projeto, marco, ano,...","ibovespa, alta, desvalorização, mmx, valorizaç...",1,0.711908,Moderada


In [93]:
DF2.tail()

,index,Col_X,Col_Y,N,Similarity,Fx_Similarity
1975,2019,"índice, pontos, ações, bolsa, alta, tóquio, fe...","tóquio, fechou, registradas, valores, bolsa, m...",1,0.814368,Forte
1976,2020,"índice, pontos, ações, bolsa, alta, tóquio, fe...","mineração, mmx, administração, conselho, opera...",1,0.806382,Forte
1977,2021,"índice, pontos, ações, bolsa, alta, tóquio, fe...","trimestre, minério, csn, milhões, ferro, ano, ...",1,0.728119,Moderada
1978,2022,"índice, pontos, ações, bolsa, alta, tóquio, fe...","vale, bilhões, mineradora, empresa, ações, aco...",1,0.732349,Moderada
1979,2023,"índice, pontos, ações, bolsa, alta, tóquio, fe...","mineração, energia, empresa, setor, brasil, ár...",1,0.677533,Moderada


In [105]:
file_B = pd.crosstab(DF2['Col_X'],
            DF2['Fx_Similarity'],
            values = DF2['Col_Y'],
            aggfunc='count', margins=False).replace(np.nan, 0).reset_index()
file_B.head(), file_B.shape

(Fx_Similarity                                              Col_X  Forte  \
 0              ano, empresa, mineração, ouro, milhões, projet...   26.0   
 1              brasil, mineração, setor, país, mineral, inova...   34.0   
 2              brazilian, machinery, brazil, program, abimaq,...   10.0   
 3              eike, cabral, batista, rio, sérgio, empresário...   19.0   
 4              empresa, bilhões, mundo, maior, mineração, min...   32.0   
 
 Fx_Similarity  Fraca  Moderada  Perfeita  
 0                1.0      16.0       1.0  
 1                1.0       8.0       1.0  
 2                1.0      33.0       0.0  
 3                1.0      24.0       0.0  
 4                1.0       8.0       3.0  ,
 (45, 5))

In [106]:
file_B = file_B[['Col_X', 'Fraca', 'Moderada', 'Forte', 'Perfeita']]
file_B.head()

Fx_Similarity,Col_X,Fraca,Moderada,Forte,Perfeita
0,"ano, empresa, mineração, ouro, milhões, projet...",1.0,16.0,26.0,1.0
1,"brasil, mineração, setor, país, mineral, inova...",1.0,8.0,34.0,1.0
2,"brazilian, machinery, brazil, program, abimaq,...",1.0,33.0,10.0,0.0
3,"eike, cabral, batista, rio, sérgio, empresário...",1.0,24.0,19.0,0.0
4,"empresa, bilhões, mundo, maior, mineração, min...",1.0,8.0,32.0,3.0


In [107]:
file_B['Total'] = file_B['Fraca'] + file_B['Moderada'] + file_B['Forte'] + file_B['Perfeita']
file_B.head()

Fx_Similarity,Col_X,Fraca,Moderada,Forte,Perfeita,Total
0,"ano, empresa, mineração, ouro, milhões, projet...",1.0,16.0,26.0,1.0,44.0
1,"brasil, mineração, setor, país, mineral, inova...",1.0,8.0,34.0,1.0,44.0
2,"brazilian, machinery, brazil, program, abimaq,...",1.0,33.0,10.0,0.0,44.0
3,"eike, cabral, batista, rio, sérgio, empresário...",1.0,24.0,19.0,0.0,44.0
4,"empresa, bilhões, mundo, maior, mineração, min...",1.0,8.0,32.0,3.0,44.0


In [108]:
file_B['Perfeita'].min(), file_B['Perfeita'].max(),

(0.0, 4.0)

In [109]:
file_B['Forte'].min(), file_B['Forte'].max(),

(0.0, 38.0)

In [110]:
file_B = file_B.rename(columns={'Col_X': 'Termos'})

In [111]:
DF3 = DF[['Contexto', 'Ano', 'Cluster Label', 'Termos']]

In [112]:
file_B = file_B.merge(DF3, on=['Termos'], how='left')
file_B.head(3)

,Termos,Fraca,Moderada,Forte,Perfeita,Total,Contexto,Ano,Cluster Label
0,"ano, empresa, mineração, ouro, milhões, projet...",1.0,16.0,26.0,1.0,44.0,Sem Similaridade por ano,2014,2
1,"brasil, mineração, setor, país, mineral, inova...",1.0,8.0,34.0,1.0,44.0,Sem Similaridade por ano,2015,3
2,"brazilian, machinery, brazil, program, abimaq,...",1.0,33.0,10.0,0.0,44.0,Sem Similaridade por ano,2017,2


In [113]:
file_B.columns

Index(['Termos', 'Fraca', 'Moderada', 'Forte', 'Perfeita', 'Total', 'Contexto',
       'Ano', 'Cluster Label'],
      dtype='object')

In [114]:
file_B = file_B[['Contexto', 'Ano', 'Cluster Label', 'Termos', 'Fraca', 'Moderada', 'Forte', 'Perfeita', 'Total']]
file_B.sort_values(by = ['Ano', 'Cluster Label'], ascending = [True, True], inplace = True)
file_B.head()

,Contexto,Ano,Cluster Label,Termos,Fraca,Moderada,Forte,Perfeita,Total
6,Sem Similaridade por ano,2013,1,"empresa, mineração, ouro, projeto, marco, ano,...",1.0,16.0,26.0,1.0,44.0
29,Sem Similaridade por ano,2013,2,"ogx, queda, mercado, ações, ibovespa, presse, ...",1.0,16.0,27.0,0.0,44.0
12,Sem Similaridade por ano,2013,3,"libertação, eln, tibete, exército, bogotá, sol...",1.0,20.0,23.0,0.0,44.0
26,Sem Similaridade por ano,2013,4,"mmx, minério, toneladas, ferro, recursos, azul...",0.0,7.0,37.0,0.0,44.0
9,Sem Similaridade por ano,2013,5,"ibovespa, alta, valorização, ganhos, brasília,...",1.0,6.0,37.0,0.0,44.0


In [115]:
file_B.to_excel(r'/drive/MyDrive/Doutorado_v2/E04_Similarity_Clusters.xlsx', index=False)